In [1]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from datetime import timedelta
from operator import itemgetter 
import time

In [207]:
data = pd.read_csv('Dataset_goal_3_time_csv.csv',sep=';')

selectedDay = 'Sunday'
weightDrivingTime = 1
weightWaitingTime = 1 - weightDrivingTime


### Calcuate average travel time from max and min travel time
city = list()
highway = list()
for row in np.arange(data.shape[0]):
    #print('Scenario I average: ',sum(list(data.iloc[row,3:5].values))/2)
    city.append(sum(list(data.iloc[row,3:5].values))/2)
    
    #print('Scenario II average: ',sum(list(data.iloc[row,6:].values))/2)
    highway.append(sum(list(data.iloc[row,6:].values))/2)
    
data['averageTimeCity'] = city
data['averageTimeHighway'] = highway

### Split Columns

daySplitted = data.Day.str.split(' - ', n = 1, expand=True)

data['Weekday'] = daySplitted[0]
data['Date'] = daySplitted[1]

dateSplitted = data.Date.str.split('/',n=2, expand = True)

data['Month'] = dateSplitted[0]
data['Day'] = dateSplitted[1]
data['Year'] = dateSplitted[2]

timeSplitted = data['Departure time'].str.split(':',n=1, expand=True)

data['Hour'] = timeSplitted[0]
data['Minute'] = timeSplitted[1]


days = list(data.Weekday.drop_duplicates())
    
### tuples(scenario, weekday, year, month, day, hour, minute | trip time)

tupledict_input = list()
N=10
for d in days:
    tempData = data[data.Weekday == d]
    for st in range(len(tempData)):
        startingTime = tempData['Departure time'].values[st]
        averageTravelTimeCity = tempData['averageTimeCity'].values[st]
        averageTravelTimeHighway = tempData['averageTimeHighway'].values[st]
        tupledict_input.append([('city', d, tempData.iloc[st,:].Year,tempData.iloc[st,:].Month,
                                tempData.iloc[st,:].Day,tempData.iloc[st,:].Hour,tempData.iloc[st,:].Minute), 
                                averageTravelTimeCity])
        tupledict_input.append([('highway', d, tempData.iloc[st,:].Year,tempData.iloc[st,:].Month,
                                tempData.iloc[st,:].Day,tempData.iloc[st,:].Hour,tempData.iloc[st,:].Minute), 
                                averageTravelTimeHighway])

times = tupledict(tupledict_input) 
relevantTimes = times.subset('*', '*')
min_value = min(relevantTimes.select())
min_day = [i for i, x in enumerate(relevantTimes) if x == min_value]

### calculate arrival time for each trip with min time
arrivalTime = list()

for t in relevantTimes.iteritems():
#     if t[1] == min_value:
#         arrivalTime.append(datetime(int(t[0][2]),int(t[0][3]),int(t[0][4]),
#                              int(t[0][5]),int(t[0][6])) + timedelta(minutes = min_value))
    arrivalTime.append(datetime(int(t[0][2]),int(t[0][3]),int(t[0][4]),
                             int(t[0][5]),int(t[0][6])) + timedelta(minutes = min_value))

goalTime = 10 * 60
arrivalTimeMinutes = [arrivalTime[x].hour * 60 + arrivalTime[x].minute for x in range(len(arrivalTime))]

m = Model()

### Create DataFrame with all possible routes and departure times
pddf = pd.DataFrame.from_dict(relevantTimes.items())

pddfSplitted = pd.DataFrame(pddf[0].tolist(), index=pddf.index)
pddfSplitted.columns=['Route','Weekday','Year','Month','Day','departureTimeHour','departureTimeMinute']
pddfSplitted['travelTime']=pddf[1]

### Add arrival time
arrivalHour = [arrivalTime[x].hour for x in range(len(arrivalTime))]
arrivalMinute = [arrivalTime[x].minute for x in range(len(arrivalTime))]

pddfSplitted['arrivalTimeHour'] = arrivalHour
pddfSplitted['arrivalTimeMinute'] = arrivalMinute

### Add waiting time
waitingTime = [goalTime - arrivalTimeMinutes[x] for x in range(len(arrivalTimeMinutes))]
pddfSplitted['waitingTime'] = waitingTime

### Keep only scenarios where arrival is before goal time
possibleRecommendations = pddfSplitted[pddfSplitted.waitingTime >= 0]
possibleRecommendations.index = range(len(possibleRecommendations))

scenarioScores = possibleRecommendations.travelTime * weightDrivingTime + possibleRecommendations.waitingTime * weightWaitingTime
recommendedRoute = possibleRecommendations.iloc[scenarioScores.idxmin()]

### variable to choose only the minimum driving time

choice = list()
for x in range(len(relevantTimes)):
    choice.append(m.addVar(vtype=GRB.BINARY))
    
temp = relevantTimes.select()
    
m.update()

m.addConstr(quicksum(choice) == 1)

m.setObjective(quicksum(temp[x] * choice[x] for x in range(len(temp))), GRB.MINIMIZE)

m.optimize()

printSolution()

Optimize a model with 1 rows, 112 columns and 112 nonzeros
Variable types: 0 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 18.0000000
Presolve removed 1 rows and 112 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 15 18 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+01, best bound 1.500000000000e+01, gap 0.0000%

------------------------------------------------------------

Shortest possible driving time: 15 minutes

Recommendation for Sunday (23.06.19):

------------------------------------------------------------

Recommended Route: highway

Departure Time: 06 : 45

Arrival Time: 07 : 00

Driving Time: 15 minutes

Waiting Time: 18

C:\Users\yubim\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [206]:
def printSolution():
    print('\n------------------------------------------------------------')
    print('\nShortest possible driving time: %g minutes' % (m.ObjVal))
    print('\nRecommendation for %s (%s.%s.%s):' % (recommendedRoute.Weekday,recommendedRoute.Day,recommendedRoute.Month,recommendedRoute.Year))
    print('\n------------------------------------------------------------')
    print('\nRecommended Route: %s' % (recommendedRoute.Route))
    print('\nDeparture Time: %s : %s' % (recommendedRoute.departureTimeHour, recommendedRoute.departureTimeMinute))
    print('\nArrival Time: 0%g : 0%g' % (recommendedRoute.arrivalTimeHour, recommendedRoute.arrivalTimeMinute))
    print('\nDriving Time: %g minutes' % (recommendedRoute.travelTime))
    print('\nWaiting Time: %g minutes' % (recommendedRoute.waitingTime))